In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [87]:
import re
from datetime import datetime
from geopy import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [88]:
locator = Nominatim(user_agent="ppcoom")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1/20)
dengue_regex = re.compile(r'([A-Za-z ]+).*\[w\/e (.+)\] \/ (.+) \/ (.+) \/ (.+) \/ (.+) \/ (.+)', re.MULTILINE)

In [4]:
df = pd.read_feather('combined_df_anomaly.feather')

In [181]:
dengue_df = df[df['disease'] == 'dengue']

In [183]:
def parse_dengue(row):
    return pd.DataFrame([(
        *row[2:],
        match[0].strip(),
        pd.NA if re.match(r'\d+ \w{3} \w{4}', re.sub(r'[^\w ]+', '', match[1])) is None 
            else datetime.strptime(re.sub(r'[^\w ]+', '', match[1]), r'%d %b %Y'),
        match[2],
        *[pd.NA if not match[i].replace(' ', '').isnumeric() else int(match[i].replace(' ', '')) for i in range(3, 7)])
        for match in dengue_regex.findall(row['content'])],
        columns=[
        *row[2:].keys(),
        'location_name',
        'dates_start',
        'serotype',
        'total_cases',
        'confirmed_cases',
        'severe_cases',
        'deaths'
    ])


In [184]:
dengue_df_parse = pd.concat([parse_dengue(row) for i, row in tqdm(dengue_df.iterrows())])


1412it [00:43, 32.57it/s] 


In [185]:
dengue_df_parse['location'] = dengue_df_parse['location_name'].progress_apply(geocode)
dengue_df_parse['point'] = dengue_df_parse['location'].progress_apply(lambda loc: tuple(loc.point) if loc else None)
dengue_df_parse[['zoom_lat', 'zoom_lon', 'altitude']] = pd.DataFrame(dengue_df_parse['point'].tolist(), index=dengue_df_parse.index)
dengue_df_parse = dengue_df_parse.drop(['location', 'point', 'altitude'], axis=1)
dengue_df_parse = dengue_df_parse.reset_index(drop=True)

100%|██████████| 788/788 [00:00<00:00, 260101.64it/s]


In [186]:
dengue_df_parse.to_feather('dengue_df.feather')

In [187]:
dengue_df = pd.read_feather('dengue_df.feather')
other_df = pd.read_feather('dataset.v1.1.feather')

In [188]:
dengue_df = dengue_df.rename({'confirmed_cases': 'cases'}, axis=1)
full_df = pd.concat([dengue_df,other_df], axis=0, ignore_index=True)

In [189]:
len(full_df)

2112

In [190]:
full_df = full_df.drop(['Unnamed: 0', 'index'], axis=1)

In [191]:
full_df.to_feather('dataset.v1.2.feather')

In [192]:
full_df.iloc[0]

id                                                                 8163513
title                                 ['Dengue/DHF update (01): Americas']
zoom_lat                                                          23.95046
zoom_lon                                                        -102.53289
zoom_level                                                               5
alert_id                                                           8163513
feed_id                                                                  1
summary                          PRO/EDR> Dengue/DHF update (01): Americas
issue_date                                             2021-02-04 06:39:21
load_date                                              2021-02-04 06:39:21
incident_date                                                          NaN
descr                    Cases by Country / Week updated / Serotype / T...
alert_tag_id                                                           NaN
dup_count                

In [196]:
full_df[(full_df['diseimerg = ase'] == 'dengue') & full_df['cases'] > 1000]

In [195]:
from datetime import datetime
time = datetime(2000, 6, 21, 19, 30, 0)
time_end = time.strftime(r'%Y-%m-%dT%H:%M:%S')

'2000-06-21T19:30:00'

In [3]:
imerg = """prod_name,GPM_3IMERGHH.06
param_short_name,precipitationCal
param_name,Multi-satellite precipitation estimate with gauge calibration - Final Run (recommended for general use)
unit,mm/hr
undef,-9.9999e+3
begin_time,2000-06-01 00:00:00
end_time,2000-06-01 07:30:00
lat,4.75
lon,0.55
lat_resolution,0.1
lat_resolution,0.1
mean,9.7633e+00
Request_time,2021-08-09 05:47:54

Timestamp (UTC),Data
2000-06-01 00:00,1.6826e+00
2000-06-01 00:30,1.8239e+00
2000-06-01 01:00,3.2667e+00
2000-06-01 01:30,1.7841e+00
2000-06-01 02:00,8.7605e-01
2000-06-01 02:30,1.8455e+00
2000-06-01 03:00,3.6081e+00"""

In [2]:
from io import StringIO

In [5]:
import pandas as pd

In [9]:
import requests
from io import StringIO

# ...
# ...
response = requests.get('https://api.giovanni.earthdata.nasa.gov/timeseries', headers=headers, params=params)

df = None
if response.status_code == 200:
	result = response.text
	with StringIO(result[result.index('Timestamp (UTC)'):]) as data:
		df = pd.read_csv(data)
else:
	print(f'Error: code {response.status_code}')

In [13]:
test.keys()

Index(['Timestamp (UTC)', 'Data'], dtype='object')

In [14]:
from datetime import datetime

In [15]:
datetime(year=2000, month=6, day=21, hour=19, minute=30, second=0)

datetime.datetime(2000, 6, 21, 19, 30)

In [1]:
import pandas as pd

In [2]:
df = pd.read_feather('dataset.v1.2.feather')

In [3]:
df.head()

,id,title,zoom_lat,zoom_lon,zoom_level,alert_id,feed_id,summary,issue_date,load_date,...,deaths,location_lat,location_lon,admin1_code,admin2_code,admin3_code,admin4_code,cases_tags,deaths_tags,dates_end
0,8163513,['Dengue/DHF update (01): Americas'],23.95046,-102.53289,5,8163513,1,PRO/EDR> Dengue/DHF update (01): Americas,2021-02-04 06:39:21,2021-02-04 06:39:21,...,0.0,39.783730,-100.445882,None,None,None,None,None,None,NaT
1,8163513,['Dengue/DHF update (01): Americas'],23.95046,-102.53289,5,8163513,1,PRO/EDR> Dengue/DHF update (01): Americas,2021-02-04 06:39:21,2021-02-04 06:39:21,...,0.0,16.825979,-88.760093,None,None,None,None,None,None,NaT
2,8163513,['Dengue/DHF update (01): Americas'],23.95046,-102.53289,5,8163513,1,PRO/EDR> Dengue/DHF update (01): Americas,2021-02-04 06:39:21,2021-02-04 06:39:21,...,0.0,10.273563,-84.073910,None,None,None,None,None,None,NaT
3,8163513,['Dengue/DHF update (01): Americas'],23.95046,-102.53289,5,8163513,1,PRO/EDR> Dengue/DHF update (01): Americas,2021-02-04 06:39:21,2021-02-04 06:39:21,...,0.0,13.800038,-88.914068,None,None,None,None,None,None,NaT
4,8163513,['Dengue/DHF update (01): Americas'],23.95046,-102.53289,5,8163513,1,PRO/EDR> Dengue/DHF update (01): Americas,2021-02-04 06:39:21,2021-02-04 06:39:21,...,8.0,15.635609,-89.898809,None,None,None,None,None,None,NaT


In [4]:
len(df)

2112

In [9]:
# from geopy import Nominatim
# from geopy.extra.rate_limiter import RateLimiter

In [10]:
# locator = Nominatim(user_agent="bruh")
# geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

In [11]:
# location

Location(Champ de Mars, Rue George Bernard Shaw, Quartier de Grenelle, Paris 15e Arrondissement, Paris, Île-de-France, France métropolitaine, 75015, France, (48.85614465, 2.297820393322227, 0.0))

In [28]:
df.loc[df['disease'] == 'dengue', ['zoom_lat', 'zoom_lon']] = df.loc[df['disease'] == 'dengue', ['location_lat', 'location_lon']].values

In [29]:
df.loc[df['disease'] == 'dengue', ['zoom_lat', 'zoom_lon']]

,zoom_lat,zoom_lon
0,39.783730,-100.445882
1,16.825979,-88.760093
2,10.273563,-84.073910
3,13.800038,-88.914068
4,15.635609,-89.898809
...,...,...
783,-1.339767,-79.366697
784,-6.869970,-75.045851
785,-34.996496,-64.967282
786,-10.333333,-53.200000


In [31]:
df = df.drop(['location_lat', 'location_lon'], axis=1).reset_index()

In [35]:
df.to_feather('dataset.v1.3.feather')

In [24]:
df.loc[df['disease'] == 'dengue', ['location_lat', 'location_lon']]

,location_lat,location_lon
0,39.783730,-100.445882
1,16.825979,-88.760093
2,10.273563,-84.073910
3,13.800038,-88.914068
4,15.635609,-89.898809
...,...,...
783,-1.339767,-79.366697
784,-6.869970,-75.045851
785,-34.996496,-64.967282
786,-10.333333,-53.200000


In [19]:
df.loc[df['disease'] == 'dengue', ['zoom_lat', 'zoom_lon']]

,zoom_lat,zoom_lon
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
783,NaN,NaN
784,NaN,NaN
785,NaN,NaN
786,NaN,NaN


In [6]:
df.iloc[1].content

'DENGUE/DHF UPDATE (01): AMERICAS\n********************************\nA ProMED-mail post\n<http://www.promedmail.org>\nProMED-mail is a program of the\nInternational Society for Infectious Diseases\n<http://www.isid.org>\n\nIn this update:\nCases in various countries\n- WHO/PAHO countries in the Americas reporting dengue cases\n- WHO epidemiological update\n\nMexico and Central America\n---\nMexico\n- National\n- Hidalgo state\n- Jalisco state\nCosta Rica\n\nCaribbean\n---\nSt Vincent and the Grenadines\n\nSouth America\n---\nBrazil\n- National\n- Rio de Janeiro state\n- Ribeirao Preto, Sao Paulo state\n- Minas Gerais state\n- West central Minas Gerais state\n- Cascavel, Parana state\n- Rio Branco, Arce state\nColombia\nParaguay\nPeru\n\n******\nCases in various countries\n- WHO/PAHO countries in the Americas reporting dengue cases\nDate: Sat 2 Jan 2021\n<http://www.paho.org/data/index.php/en/mnu-topics/indicadores-dengue-en/dengue-nacional-en/252-dengue-pais-ano-en.html>\n\n\nReported 